In [213]:
import re
import pandas as pd
pd.options.display.max_rows = None
import datetime
import time

In [2]:
# 開啟檔案
def open_brd_file(filepath):
    f = open(filepath)
    return f

# 找到ballname
def findBallName(ballName, string):
    result = re.findall(f"(UCPU1\.{ballName}\,|UCPU1\.{ballName}$)", string, re.I)
    return result

In [453]:
#step1: 開啟檔案
filepath = "ORB_ADL_LPDDR5_1125b.txt"
try:
    f = open_brd_file(filepath)
    print(f"step1: Open file. (File name: {filepath}) -DONE")
except:
    print(f"File is not exit, check file name.")

rawData = []
for i in f:
    row = i.replace("\n","")#去掉換行符號
    rawData.append(row)

step1: Open file. (File name: ORB_ADL_LPDDR5_1125b.txt) -DONE


In [454]:
#找出個$NETS並清除不需要的資料
keyIDX = []
for i in rawData:
    if re.findall(f"\$(NETS|END)", i , re.I):
#         print(re.findall(f"\$.*", i, re.I))
        keyIDX.append(rawData.index(i))
rawData2 = rawData[keyIDX[0] +1: keyIDX[1]]

In [455]:
# 把TXT整理成dictionary
start_time = time.time()
netName, netData = "", ""
netInfo = {}
clearData = {}
# 處理txt檔 分好net name 跟 GPIO資料
for line in rawData2:
#     print(re.findall(f".+\;", line))
    if re.findall(f".+\;", line):
        netName = re.findall(f".+\;", line)[0].replace(";", "")#分號前是net name
        
        #處理 net data 
        netData = re.findall(f"\;.+", line)[0].replace(";", "")#分號後是net data
        netData = netData.split()
        netData = ",".join(netData)


        netInfo = { netName : netData }
        clearData.update(netInfo)
    else:
        line = ",".join(line.split())#用空格白切割 再用,分割
        
        if clearData.__contains__(netName):
            clearData[netName] += line
        else:
            print("erroe", line)

end_time = time.time()
print ("執行時間2：", (end_time - start_time))

執行時間2： 0.05155444145202637


In [474]:
# 叫出intel GPIO template 
data = pd.read_excel('GPIO2020.xlsx')
CPUBallName = data.iloc[:,0].tolist()

In [471]:
data["netName"] = ""

In [472]:
for netName, netData in clearData.items():
#     print(netData)
    for ballName in CPUBallName:
        result = findBallName(ballName, netData)
#         print(result)
        if result:
            if data.loc[data[data["CPU Ball Name"]==f"{ballName}"].index.tolist()[0], "netName" ] == "":
#                 print("yes")
                data.loc[data[data["CPU Ball Name"]==f"{ballName}"].index.tolist()[0], "netName" ] = netName
            else:
            
                print("重複")
                data.loc[data[data["CPU Ball Name"]==f"{ballName}"].index.tolist()[0], "netName" ] += netName


In [473]:
elsxPath = f'GPIO mapping_20201228.xlsx'
write = pd.ExcelWriter(elsxPath)

# save_excel(write, dfFunction, "dfFunction")
data.to_excel(write, sheet_name=f'data', index=False)

write.save()